In [ ]:
import pandas as pd
%config ZMQInteractiveShell.ast_node_interactivity='all'
%pprint
import warnings 
warnings.filterwarnings('ignore')

In [28]:
import re
import numpy as np
import pandas as pd

## 读取文本

In [3]:
df_test = pd.read_csv('./data/test.csv',sep="###__###",names=['ID','Query_List'])
df_train = pd.read_csv('./data/train.csv',sep='###__###',names=['ID', 'Age', 'Gender', 'Education', 'Query_List'])

In [4]:
df_test.head()
df_train.head(10)

,ID,Query_List
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...
5,5BB424C7298B2CB67C368DDB73D4D336,4,2,0,钢琴曲欣赏100首\t一个月的宝宝眼睫毛那么是黄色\t宝宝右眼有眼屎\t小儿抽搐怎么办\t剖...
6,0D58894CD481D4B3039000661665C0DB,3,2,3,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...
7,0BA56489237BC91353300BE44298C6D8,6,1,0,昂的笔顺\t三年级上册数学口算\t牛的英语怎么读\t抓木机\t60米差不多跑多长时间\t似乎...
8,3D39C1B9312105E030B039CCDCD79611,1,1,5,李明霖在哪直播\t英文翻译\t薄荷籽\t为什么搜狐视频不能缓存\t有一首歌女生唱的歌词是冬天...
9,93174489E2E708BF6F797DC69F1531A8,1,1,5,bft20\t龙王传说\t长治到成都可以怎么坐火车\t龙王传说txt\t2 4 8 16老司...


## 查看用户数据,看数据均衡

In [5]:
df_train.info() # 数据没有缺失值
df_train[['Age','Gender','Education']].agg(['mean','sum','median','count']) 
for i in ['Age','Gender','Education']:
    df_train[i].value_counts()

# 数据在Age=1的比较多, Gender=1比较多,education在5比较多,可以看出样本总体不均衡,所以训练模型的时候需要切分

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
ID            100000 non-null object
Age           100000 non-null int64
Gender        100000 non-null int64
Education     100000 non-null int64
Query_List    100000 non-null object
dtypes: int64(3), object(2)
memory usage: 3.8+ MB


,Age,Gender,Education
mean,2.08231,1.38714,3.90333
sum,208231.00000,138714.00000,390333.00000
median,2.00000,1.00000,4.00000
count,100000.00000,100000.00000,100000.00000


1    38996
2    26744
3    18529
4    10654
5     2922
0     1666
6      489
Name: Age, dtype: int64

1    56976
2    40869
0     2155
Name: Gender, dtype: int64

5    37107
4    28148
3    18858
0     9280
6     5693
2      560
1      354
Name: Education, dtype: int64

In [22]:
np.mean(df_train['Query_List'].agg(lambda x: len(x)))

1232.69503

## 清洗数据

In [23]:
df_train

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...
...,...,...,...,...,...
99995,E797FFCDCAF3899AB4D17B61170D8BFF,1,1,5,梦三生\t逆行天后漫画结局\t英雄联盟角色介绍大全\t韩剧网最新韩国电视剧\t119宣传语是...
99996,E06375F7D092ABDE78C2D79E4725D6B0,1,1,5,中国军队配枪\t女生\t央视版权问题\t重庆\t一次性手术刀\t精神枷锁\t人民检察官\t舆...
99997,D55119CB0B9366B20974522B58C00912,2,2,5,英文翻译\t幼儿园面试讲课\t学前教育书第二版\tshock to\t幼儿园教师资格证面试讲...
99998,EB4DBBD602C6459A19A77F09035E170C,3,2,3,哈尔滨祖研中医院地址\t指甲盖侧面的肉怎么是白色\t补骨质有副作用吗\t哈尔滨去呼兰\t黑龙...


### 读取停用词

In [62]:
stopwords = pd.read_csv('./data/stop.txt',sep='\t',index_col=False,quoting=3,encoding='utf-8',names=['stopwords'])
stopwords = stopwords['stopwords'].values

### 清洗词函数

In [67]:
# 1 思路: 用正则匹配有http或者www, 看看都是什么类型的,然后用离散型数值表示
pattern_http = '((http|ftp|https):\/\/[\w\-_]+[\w\-_]+[\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#]?)'
def http_cov(text):
    text = text.replace('ｗｗｗ．','www.').replace('．ｃｏｍ','.com').replace('\t','')
    text = re.sub(pattern_http,"WEB",text)
    return text
# 2 思路: 用正则匹配有邮箱地址, 然后用离散型数值表示
def email_cov(text):
    text = re.sub('([A-Za-z0-9\u4e00-\u9fa5]+@[a-zA-Z0-9_-]+(\.[a-zA-Z0-9_-]+))','EML',text)
    return text
#to do 可以把各个专业领域的专业用词导入,做分词,使用jieba.load_userdict(file_name), jieba.cut(ori, HMM=False),
# 词典格式和dict.txt一样，一个词占一行；每一行分三部分，一部分为词语，另一部分为词频，最后为词性（可省略），用空格隔开
# 宜信普惠 7
# 宜信 10
# 极速模式 20
# 汽车方面可以加入结巴,品牌对人群区分有影响
# 分词与词性过滤 (只保留n,v,j)

In [68]:
df_train['Query_List'] = df_train['Query_List'].apply(http_cov).apply(email_cov)

In [76]:
import jieba
import jieba.analyse
df_train['Query_List'] = df_train['Query_List'].apply(lambda x: [i for i in jieba.lcut(x) if i not in stopwords.tolist()])


In [77]:
df_train

,ID,Age,Gender,Education,Query_List
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,"[柔和, 双沟, 女生, 中财网, 首页, , 财经, WEB, 周公, 解梦, 大全, ..."
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"[广州, 厨宝, 烤箱, 世情, 薄, 人情, 恶, 雨送, 黄昏, 花易落, 晓, 风干,..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,"[钻石, 之泪, 耳机, 盘锦, 沈阳, 旅顺, 公交, 辽宁, 阜新, 车牌, baidu..."
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,"[受欢迎, 狗狗, 排行榜, 舶, 读, 场景, 描, , 写, 范例, 三维, 绘图, ..."
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,"[干槽症, 自愈, 太太, 万岁, 叶, 舒心, 美国, 干槽症, 右, 眼皮, 跳, 怎么..."
...,...,...,...,...,...
99995,E797FFCDCAF3899AB4D17B61170D8BFF,1,1,5,"[梦, 三生, 逆行, 天后, 漫画, 结局, 英雄, 联盟, 角色, 介绍, 大全, 韩剧..."
99996,E06375F7D092ABDE78C2D79E4725D6B0,1,1,5,"[中国, 军队, 配枪, 女生, 央视, 版权, 重庆, 一次性, 手术刀, 精神枷锁, 检..."
99997,D55119CB0B9366B20974522B58C00912,2,2,5,"[英文翻译, 幼儿园, 面试, 讲课, 学前教育, 书, 版, shock, , 幼儿园,..."
99998,EB4DBBD602C6459A19A77F09035E170C,3,2,3,"[哈尔滨, 祖研, 中, 医院地址, 指甲盖, 侧面, 肉, 白色, 补, 骨质, 副作用,..."


In [50]:
# import jieba
# import jieba.analyse
# sentences = jieba.lcut(df_train['Query_List'][0])
# # jieba.analyse.extract_tags(sentences,topK=20)
# sentences = ' '.join(sentences)
# sentences.strip()
# jieba.analyse.extract_tags(sentences,topK=20)

In [106]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,precision_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

In [93]:
df_train['Query_List'] = df_train['Query_List'].apply(lambda x : ' '.join(x))

In [102]:
X = df_train['Query_List'].values.tolist()
Y = df_train['Age'].values.tolist()
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=22)
vec = TfidfVectorizer(analyzer='word',max_features=10000)
vec.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
def stratifiedkfold_cv(x,y,clf_class,shuffle=True,n_folds=5,**kwargs):
    stratifiedk_fold=StratifiedKFold(n_splits=n_folds,shuffle=shuffle)
    y_pred = y[:]
    for train_index,test_index in stratifiedk_fold.split(x,y):
        X_train,X_test = x[train_index],x[test_index]
        y_train= y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred
NB = MultinomialNB
svm = SVC
BR = BernoulliNB
for i in [NB,svm,BR]:
    print(precision_score(Y,stratifiedkfold_cv(vec.transform(X),np.array(Y),i),average='macro'))

0.26819080442394533


In [ ]:
from gensim.models import word2vec